In [1]:
import numpy as np

In [2]:
import pickle as pk

In [3]:
T = 96
dT = 4
num_cluster = 1600

In [4]:
data = dict({})

for m in range(10, 12):
    for d in range(1, 29):
        filename = '/data/fan/UsersInOsakaProcessed/201210{:02d}_interp.pk'.format(d)
        print(filename)
        with open(filename, 'rb') as f:
            data[(m, d)] = pk.load(f)

/data/fan/UsersInOsakaProcessed/20121001_interp.pk
/data/fan/UsersInOsakaProcessed/20121002_interp.pk
/data/fan/UsersInOsakaProcessed/20121003_interp.pk
/data/fan/UsersInOsakaProcessed/20121004_interp.pk
/data/fan/UsersInOsakaProcessed/20121005_interp.pk
/data/fan/UsersInOsakaProcessed/20121006_interp.pk
/data/fan/UsersInOsakaProcessed/20121007_interp.pk
/data/fan/UsersInOsakaProcessed/20121008_interp.pk
/data/fan/UsersInOsakaProcessed/20121009_interp.pk
/data/fan/UsersInOsakaProcessed/20121010_interp.pk
/data/fan/UsersInOsakaProcessed/20121011_interp.pk
/data/fan/UsersInOsakaProcessed/20121012_interp.pk
/data/fan/UsersInOsakaProcessed/20121013_interp.pk
/data/fan/UsersInOsakaProcessed/20121014_interp.pk
/data/fan/UsersInOsakaProcessed/20121015_interp.pk
/data/fan/UsersInOsakaProcessed/20121016_interp.pk
/data/fan/UsersInOsakaProcessed/20121017_interp.pk
/data/fan/UsersInOsakaProcessed/20121018_interp.pk
/data/fan/UsersInOsakaProcessed/20121019_interp.pk
/data/fan/UsersInOsakaProcessed

In [5]:
global_dist = np.zeros([num_cluster, T], dtype=np.float32)

In [6]:
uid_set = set([])
for d in data:
    uid_set = uid_set | set(list(data[d].keys()))

In [7]:
local_dist = dict({})
for uid in uid_set:
    local_dist[uid] = np.zeros([num_cluster, T], dtype=np.float32)

In [8]:
for d in data:
    print(d)
    for uid in data[d]:
        for t in range(2 * dT, T):
            local_dist[uid][data[d][uid][t], t] += 1

(10, 1)
(10, 2)
(10, 3)
(10, 4)
(10, 5)
(10, 6)
(10, 7)
(10, 8)
(10, 9)
(10, 10)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(10, 16)
(10, 17)
(10, 18)
(10, 19)
(10, 20)
(10, 21)
(10, 22)
(10, 23)
(10, 24)
(10, 25)
(10, 26)
(10, 27)
(10, 28)
(11, 1)
(11, 2)
(11, 3)
(11, 4)
(11, 5)
(11, 6)
(11, 7)
(11, 8)
(11, 9)
(11, 10)
(11, 11)
(11, 12)
(11, 13)
(11, 14)
(11, 15)
(11, 16)
(11, 17)
(11, 18)
(11, 19)
(11, 20)
(11, 21)
(11, 22)
(11, 23)
(11, 24)
(11, 25)
(11, 26)
(11, 27)
(11, 28)


In [9]:
for uid in local_dist:
    global_dist += local_dist[uid]

In [10]:
global_dist += 0.5

In [11]:
global_dist /= np.sum(global_dist, axis=0)

In [12]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [13]:
import os

In [15]:
metrics_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInOsakaProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)

    avg_loss = 0.0
    mrr = 0.0
    avg_rank = 0.0
    acc5 = 0.0
    
    with open(filename, 'rb') as f:
        data = pk.load(f)
        
    cnt = 0
    for uid in data:
        if uid in local_dist:
            if np.random.ranf() < 0.2:
                dist = 0.6 * (local_dist[uid] + 1e-4) / (np.sum(local_dist[uid], axis=0) + 1e-4) + 0.4 * global_dist
                for t in range(2 * dT, T):
                    cnt += 1
                    avg_loss += -np.log(dist[data[uid][t], t])
                    rank = np.where((np.argsort(-dist[:, t]).T - data[uid][t]).T == 0)[0] + 1
                    mrr += np.mean(1 / rank)
                    avg_rank += np.mean(rank)
                    acc5 += np.mean(rank <= 5)
                
    
    avg_loss /= cnt
    mrr /= cnt
    avg_rank /= cnt
    acc5 /= cnt
    print(y, m, d, avg_loss, mrr, avg_rank, acc5)
    metrics_dict[(y, m, d)] = (avg_loss, mrr, avg_rank, acc5)

/data/fan/UsersInOsakaProcessed/20121201_interp.pk
2012 12 1 3.201041660017359 0.5694667804498629 122.9463040339181 0.7107591631079296
/data/fan/UsersInOsakaProcessed/20121202_interp.pk
2012 12 2 3.2411708931938237 0.5559965631233911 122.86028448437207 0.7096221904616066
/data/fan/UsersInOsakaProcessed/20121203_interp.pk
2012 12 3 2.6822165904493747 0.6559178640606709 93.66255155317656 0.7825278606528606
/data/fan/UsersInOsakaProcessed/20121204_interp.pk
2012 12 4 2.6724597321057035 0.6590887257242641 93.35111713518731 0.783449960789045
/data/fan/UsersInOsakaProcessed/20121205_interp.pk
2012 12 5 2.73718717325361 0.6479624966590477 96.61881164148863 0.7744536068651113
/data/fan/UsersInOsakaProcessed/20121206_interp.pk
2012 12 6 2.725031087688669 0.6522356196704466 97.63639500214025 0.776066805414102
/data/fan/UsersInOsakaProcessed/20121207_interp.pk
2012 12 7 2.850244231765368 0.6340828847261386 106.04140939744264 0.7588164885852747
/data/fan/UsersInOsakaProcessed/20121208_interp.pk
20

2013 1 30 2.8258202128318763 0.6366687350428926 104.40222934959777 0.7626934469039732
/data/fan/UsersInOsakaProcessed/20130131_interp.pk
2013 1 31 2.792922611853636 0.6423595089496071 101.81556536511513 0.7661080205619323
/data/fan/UsersInOsakaProcessed/20130201_interp.pk
2013 2 1 2.8582796116166063 0.6334190572590143 106.31538765960408 0.7566711895070104
/data/fan/UsersInOsakaProcessed/20130202_interp.pk
2013 2 2 3.2234867659681883 0.5660060617272105 126.82019729166268 0.708973280033663
/data/fan/UsersInOsakaProcessed/20130203_interp.pk
2013 2 3 3.2828644107519724 0.5499927089476203 125.29743474857342 0.7050063860906824
/data/fan/UsersInOsakaProcessed/20130204_interp.pk
2013 2 4 2.847120624689355 0.6335895665751031 105.26667228180509 0.7621801710745503
/data/fan/UsersInOsakaProcessed/20130205_interp.pk
2013 2 5 2.8163684438644356 0.6389958416782223 104.8898140783456 0.7639509029447387
/data/fan/UsersInOsakaProcessed/20130206_interp.pk
2013 2 6 2.908712183687036 0.6242576804931311 110.

In [17]:
with open('./results_osaka/pmm_metrics.pk', 'wb') as f:
    pk.dump(metrics_dict, f)